In [1]:
import pandas as pd   
from tqdm import tqdm
tqdm.pandas()
import ast

In [2]:
df = pd.read_csv("../dataset/also_buy_formatted.csv")

In [3]:
df.head(2)

,asin,category,title,description,brand,also_buy,also_view,available_also_buy,available_also_buy_title,available_also_buy_description
0,0853347867,"['Grocery & Gourmet Food', 'Cooking & Baking',...",Trim Healthy Mama Xylitol,"['Shipped from UK, please allow 10 to 21 busin...",NaN,"['B01898YHXK', 'B01BCM6LAC', 'B00Q4OL47O', 'B0...",[],"['B01898YHXK', 'B00Q4OL47O', 'B00Q4OL2F8', 'B0...","[array(['Trim Healthy Mama Gluten Free, Low Gl...","[array(['[""-Gluten-Free -NO-GMO\'s -High Fiber..."
1,4639725043,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",Lipton Yellow Label Tea (loose tea) - 450g,['Lipton Yellow Label Tea use only the finest ...,Lipton,"['B00886E4K0', 'B00CREXSHY', 'B001QTRGAQ', 'B0...","['B00CREXSHY', 'B001QTRGAQ', 'B000JSQK70', 'B0...","['B00886E4K0', 'B001QTRGAQ', 'B002EYZM4O', 'B0...","[array(['Alwazah Tea, Pure Ceylon, 14.1 Ounce'...","[array(['[""Alwazah Tea\'s popularity is mainly..."


In [4]:
df.shape

(83294, 10)

In [5]:
# delete all the null values
df = df.dropna(subset=['also_buy'])

In [6]:
df.shape

(83294, 10)

In [7]:
# delete the duplicates
df = df.drop_duplicates(subset=['asin', 'title'])

In [8]:
df.shape

(82270, 10)

In [9]:
# delete 2 columns
df = df.drop(columns=['brand', 'also_view'])

In [10]:
df.shape

(82270, 8)

In [11]:
df["category"] = df["category"].apply(lambda x: ast.literal_eval(x))
df["also_buy"] = df["also_buy"].apply(lambda x: ast.literal_eval(x))

In [12]:
# preprocess text for word2vec
# remove all punctuation
df["description"] = df["description"].str.replace('[^\w\s]','')
df["description"] = df["description"].apply(lambda x: x.lower())

In [14]:
print(df["description"][2])

lipton yellow label teabags uses a new way to make tea to lock in as much of the tea taste and aroma as possible we developed an innovative new process we press fresh high quality tea leaves to release their natural essence and we have developed a new blend with this essence enjoy a cup of tea with perfect color and taste that reinvigorates you shipping weight 1400 keywords search terms black tea bags lipton tea bags statements regarding dietary supplements have not been evaluated by the fda and are not intended to diagnose treat cure or prevent any disease or health condition


In [15]:
# remove fda warnings
df["description"] = df["description"].str.replace('statements regarding dietary supplements have not been evaluated by the fda and are not intended to diagnose treat cure or prevent any disease or health condition','')

In [17]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [19]:
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

asin                              0
category                          0
title                             0
description                       0
also_buy                          0
available_also_buy                0
available_also_buy_title          0
available_also_buy_description    0
dtype: int64

In [21]:
nlp = spacy.load("en_core_web_sm") # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.